<a href="https://colab.research.google.com/github/akhilss99/NLP/blob/main/Sentiment_Analysis_using_Bert_Twitter_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Akhil S S
###AM.SC.P2CSC20070
####Sentiment Analysis using Twitter Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle_dataset'

In [ ]:
%cd '/content/drive/MyDrive/MCA - Sem 3/NLP Lab /Sentiment Analysis'

/content/drive/MyDrive/MCA - Sem 3/NLP Lab /Sentiment Analysis


Kaggle Dataset Download

In [ ]:
!kaggle datasets download -d arkhoshghalb/twitter-sentiment-analysis-hatred-speech

  0% 0.00/1.89M [00:00<?, ?B/s]
100% 1.89M/1.89M [00:00<00:00, 56.6MB/s]


In [ ]:
!ls

'Sentiment Analysis using Bert - Twitter Dataset.ipynb'
 twitter-sentiment-analysis-hatred-speech.zip


In [ ]:
!unzip twitter-sentiment-analysis-hatred-speech.zip

Archive:  twitter-sentiment-analysis-hatred-speech.zip
  inflating: test.csv                
  inflating: train.csv               


Importing Statements

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [60]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 7.4 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 3.3 MB 26.6 MB/s 
     |████████████████████████████████| 636 kB 75.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [76]:
import pandas as pd
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
regex = RegexpTokenizer(r"\w+", gaps=True)
from sklearn.model_selection import train_test_split
import torch
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

###Metadata :
 
 Twitter Dataset
 
 1 ---> racist/sexist tweets

 0 ---> not racist/sexist tweets

Data Analysis & Preprocessing

In [30]:
train_data = pd.read_csv('/content/drive/MyDrive/MCA - Sem 3/NLP Lab /Sentiment Analysis/train.csv', index_col='id')
train_data.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


In [31]:
test_data = pd.read_csv('/content/drive/MyDrive/MCA - Sem 3/NLP Lab /Sentiment Analysis/test.csv', index_col='id')
test_data.head()

,tweet
id,
31963,#studiolife #aislife #requires #passion #dedic...
31964,@user #white #supremacists want everyone to s...
31965,safe ways to heal your #acne!! #altwaystohe...
31966,is the hp and the cursed child book up for res...
31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [32]:
train_data.shape

(31962, 2)

In [33]:
test_data.shape

(17197, 1)

In [34]:
train_data['label'].unique()

array([0, 1])

In [35]:
non_off  = len(train_data.query('label==0'))
off = len(train_data.query('label==1'))

In [36]:
print("Number of Offensive tweets : ",off)
print("Number of non-offensive tweets : ",non_off)

Number of Offensive tweets :  2242
Number of non-offensive tweets :  29720


Data Preprocessing

In [47]:
def preprocessing(input, start, end):
  tweet = input['tweet']
  final_tweet = pd.DataFrame()
  temp = []
  for i in range(start,end):
    preprocessed_tweet = re.sub('ð','',tweet[i])
    preprocessed_tweet = stemmer.stem(preprocessed_tweet)
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    preprocessed_tweet = tokenizer.tokenize(preprocessed_tweet)
    preprocessed_tweet = [word for word in preprocessed_tweet if word not in stopwords]
    preprocessed_tweet = ' '.join(preprocessed_tweet)
    temp.append(preprocessed_tweet)
  final_tweet['tweet_after_preprocess'] = temp
  return final_tweet

In [48]:
prepro_train = preprocessing(train_data, 1, 31963)
prepro_test = preprocessing(test_data, 31963, 49160) #preprocessing the datasets

In [49]:
prepro_train.index = range(1,len(prepro_train)+1) #incrementing the index from 0 to 1
prepro_train.head() 

,tweet_after_preprocess
1,user father dysfunctional selfish drags kids d...
2,user user thanks lyft credit use cause offer w...
3,bihday majesti
4,model love u take u time ur
5,factsguide society motiv


In [50]:
prepro_test.index = range(1,len(prepro_test)+1) #incrementing the index from 0 to 1
prepro_test.head()

,tweet_after_preprocess
1,studiolife aislife requires passion dedication...
2,user white supremacists want everyone see new ...
3,safe ways heal acne altwaystoheal healthy healing
4,hp cursed child book reservations already yes ...
5,3rd bihday amazing hilarious nephew eli ahmir ...


In [51]:
prepro_train['label'] = train_data['label'] 

In [52]:
prepro_train.head()

,tweet_after_preprocess,label
1,user father dysfunctional selfish drags kids d...,0
2,user user thanks lyft credit use cause offer w...,0
3,bihday majesti,0
4,model love u take u time ur,0
5,factsguide society motiv,0


In [53]:
len(prepro_train['label'])

31962

In [54]:
print(len(prepro_train['tweet_after_preprocess'])==len(train_data['tweet'])) #checking the size of the dataset after preprocessing
print(len(prepro_test['tweet_after_preprocess'])==len(test_data['tweet']))

True
True


Splitting the data for training and test

In [102]:
X = prepro_train['tweet_after_preprocess'].values.tolist()[0:2000] #taking 2000 datasets for training
Y = prepro_train['label'].values.tolist()[0:2000]

In [103]:
import collections
from collections import Counter
print(Counter(Y)) #checking for class imbalance

Counter({0: 1855, 1: 145})


In [104]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size=0.2) #splitting dataset for training and testing

In [105]:
len(X_train),len(Y_train)

(1600, 1600)

In [106]:
print(Counter(Y_train))

Counter({0: 1484, 1: 116})


Tokenizer & Model Import

In [77]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') #bert tokenizer
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)  #bert model

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [107]:
train_encodings = tokenizer(X_train, truncation=True, padding=True) #encoding for the twitter text training data
val_encodings = tokenizer(X_test, truncation=True, padding=True) #encoding for the twitter text test data
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),Y_train)) #combining the training feature encodings and labels into a dataset
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), Y_test)) #combining the testing or validation feature encodings and labels into a dataset

In [108]:
type(model)

transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification

* Model Build & Training - approx 5 mins for a single batch
* Final accuracy recorded - 98%

In [109]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08) #adam optimizer from keras inorder to compile the model
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy']) #model compiling prior to the training
model.fit(train_dataset.shuffle(100).batch(10), 
          epochs=2,
          batch_size=10,
          validation_data=val_dataset.shuffle(100).batch(10)) #inputing the training and validation data set

Epoch 1/2
160/160 [==============================] - 287s 2s/step - loss: 0.1286 - accuracy: 0.9556 - val_loss: 0.1217 - val_accuracy: 0.9600
Epoch 2/2
160/160 [==============================] - 277s 2s/step - loss: 0.0574 - accuracy: 0.9812 - val_loss: 0.1299 - val_accuracy: 0.9675


In [110]:
model.save_pretrained("./sentimentanalysisfortwitter") #saving the model
sentiment_model = TFDistilBertForSequenceClassification.from_pretrained("./sentimentanalysisfortwitter")

Some layers from the model checkpoint at ./sentimentanalysisfortwitter were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentimentanalysisfortwitter and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing the test data for Prediction

In [112]:
prepro_test

,tweet_after_preprocess
1,studiolife aislife requires passion dedication...
2,user white supremacists want everyone see new ...
3,safe ways heal acne altwaystoheal healthy healing
4,hp cursed child book reservations already yes ...
5,3rd bihday amazing hilarious nephew eli ahmir ...
...,...
17193,thought factory left right polarisation trump ...
17194,feeling like mermaid hairflip neverready forma...
17195,hillary campaigned today ohio omg amp used wor...
17196,happy work conference right mindset leads cult...


In [145]:
def predict(test,start,end):
  temp = []
  for i in range(start,end):
    predict_input = tokenizer.encode(test[i],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
    tf_output = sentiment_model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    labels = [0,1]
    label = tf.argmax(tf_prediction, axis=1)
    label = label.numpy()
    temp.append(label[0])
  return temp

In [146]:
prediction = prepro_test['tweet_after_preprocess'].values.tolist()[0:1000]
Y_pred = predict(prediction,0,1000)

In [147]:
final_prediction = pd.DataFrame()
final_prediction['tweet'] = prediction
final_prediction['label'] = Y_pred

In [148]:
final_prediction.head()

,tweet,label
0,studiolife aislife requires passion dedication...,0
1,user white supremacists want everyone see new ...,1
2,safe ways heal acne altwaystoheal healthy healing,0
3,hp cursed child book reservations already yes ...,0
4,3rd bihday amazing hilarious nephew eli ahmir ...,0


In [142]:
prediction[1]

'user white supremacists want everyone see new â birdsâ movie â hereâ'

In [154]:
predict_input = tokenizer.encode(prediction[1],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf") #encoding of the sentence 

tf_output = sentiment_model.predict(predict_input)[0] #output from the bertmodel
tf_prediction = tf.nn.softmax(tf_output, axis=1) #applying softmax fn
labels = [0,1] 
label = tf.argmax(tf_prediction, axis=1) #return largest value
label = label.numpy() #convertion to numpy array
print(labels[label[0]]) 

1


In [149]:
tf_output

array([[-1.6964176,  1.5793489]], dtype=float32)

In [155]:
label

array([1])